In [2]:
import sys
import os 
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import warp_ERA5_to_reference
import pvlib
from pvlib import solarposition
from pvlib.irradiance import get_total_irradiance
from pvlib.location import Location

In [4]:
# check if downloaded ERA5 variables acutally contain more than just one value for the study period
base_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'
variables = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

In [5]:
year = 2019
month = 'July'
doy = 26
comp = 'maxLST'
tempOut = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/'
file = [file for file in getFilelist(os.path.join(base_path, 'surface_solar_radiation_downward_clear_sky'), '.grib') if str(year) in file and str(int(MONTH_TO_02D[month])) in file][0]

In [6]:
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_FORCE_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_FORCE_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_FORCE_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_FORCE_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_FORCE_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326
s2_pathbase = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump2/ff3f2c872c08977466e5a8dc306d2d2aabc77ad995b0716c30d1c57d0004ebfd/'

In [7]:
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/{year}/Daily_AcqTime_{comp}_{year}_{month}.tif'

S2_file = [file for file in getFilelist(s2_pathbase, 'vrt', deep=True) if 'S2' in file][0]
# warp datasets needed for calculations to the spatial extent of the sharpened LST
LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near')
dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')

slope_path = slope_sub
aspect_path = aspect_sub
dem_path = dem_sub
lat_path = lat_sub
lon_path = lon_sub

era_ds = warp_ERA5_to_reference(grib_path=file, reference_path=LST_acq_spatial_sub)
bandNumber = era_ds.RasterCount
era_time = [pd.Timestamp(era_ds.GetRasterBand(i+1).GetDescription()) for i in range(bandNumber)]

# open and load the acquisition raster from LST
LST_acq_ds = checkPath(LST_acq_spatial_sub)
arr = LST_acq_ds.GetRasterBand(doy).ReadAsArray()
arr = arr.astype(np.int64)
arr_flat = arr.ravel()
arr_ts = pd.to_datetime(arr_flat, unit='s')
arr_up = pd.Series(arr_ts).dt.ceil('h')
arr_up = arr_up.values.reshape(arr.shape)
arr_min = pd.Series(arr_ts).dt.minute
arr_min = arr_min.values.reshape(arr.shape)

# get the relevant bands of era5 dataset (i.e. those bands, that reflect the upper and lower )
bands = []
for d1 in np.unique(arr_up):
     for count, e5 in enumerate(era_time):
          if d1 == e5:
               bands.append(count)
# load the era5 variable acquisition-wise into 2D numpy array 
# load the era5 variable acquisition-wise into 2D numpy array 
arrL = []
for b in bands: 
     # compare the time of LST composite with 
     mask = arr_up == era_time[b]
     # load the band that holds the observation just after LST acquisition
     after = era_ds.GetRasterBand(b).ReadAsArray()
     # load the band that holds the observation just prior to LST acquisition
     before = era_ds.GetRasterBand(b-1).ReadAsArray()
     # calculate the linearly interpolated values at the minute of acquisition
     vals_interpolated = before - (before - after) * (np.array(arr_min, dtype=np.float16) / 60) # in J/m²
     # convert to W/m²
     vals_watt = vals_interpolated /3600
     arrL.append(vals_watt * mask)
block = np.dstack(arrL)

block[block == 0] = np.nan
ssrd_watt = np.nanmax(block, axis = 2)

/data/Aldhani/users/potzschf/conda/envs/cds_era5/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
# make a mask of the nan values, as the output of pvlib.irradiance.get_total_irradiance does skip them which hinders rebuild to 2D
valid_mask = np.isfinite(ssrd_watt)
arr_ts_masked = arr_ts[valid_mask.ravel()]

# load slope, aspect, lat, lon, dem
ds = checkPath(slope_path)
slope = ds.GetRasterBand(1).ReadAsArray()
slope_flat = slope.ravel()
slope_flat_masked = slope_flat[valid_mask.ravel()]

ds = checkPath(aspect_path)
aspect = ds.GetRasterBand(1).ReadAsArray()
aspect_flat = aspect.ravel()
aspect_flat_masked = aspect_flat[valid_mask.ravel()]

ds = checkPath(dem_path)
dem = ds.GetRasterBand(1).ReadAsArray()
dem_flat = dem.ravel()
dem_flat_masked = dem_flat[valid_mask.ravel()]

ds = checkPath(lat_path)
lat = ds.GetRasterBand(1).ReadAsArray()
lat_flat = lat.ravel()
lat_flat_masked = lat_flat[valid_mask.ravel()]

ds = checkPath(lon_path)
lon = ds.GetRasterBand(1).ReadAsArray()
lon_flat = lon.ravel()
lon_flat_masked = lon_flat[valid_mask.ravel()]

# get solar viewing conditions for acquisition time
solpos = solarposition.get_solarposition(
    time=arr_ts_masked,       # vector of timestamps
    latitude=lat_flat_masked, # vector of latitudes (same length as time or broadcastable)
    longitude=lon_flat_masked,
    altitude=dem_flat_masked
)

# calculate extraterrestrial radiation (horizontal)
dni_extra = pvlib.irradiance.get_extra_radiation(arr_ts_masked)

# Compute clearness index
ssrd_watt_flat = ssrd_watt.ravel()
ssrd_watt_flat_masked = ssrd_watt_flat[valid_mask.ravel()]
ghi = ssrd_watt_flat_masked
# cos_zenith = np.cos(np.radians(solpos['zenith'].values[0]))
# ghi_clear = dni_extra.values[0] * cos_zenith
# kt = ghi / ghi_clear


# Decompose GHI to DNI and DHI using Erbs model
dni_dhi = pvlib.irradiance.erbs(ghi, solpos['zenith'], e5)
dni, dhi = dni_dhi['dni'], dni_dhi['dhi']


# compute radiation on the tilted terrain

irradiance_tilted = get_total_irradiance(
    surface_tilt=slope_flat_masked,
    surface_azimuth=aspect_flat_masked,
    dni=dni,
    ghi=ghi,
    dhi=dhi,
    solar_zenith=solpos['zenith'],
    solar_azimuth=solpos['azimuth']
)

# bring back to 2D
poa_global_arr = np.full(slope.shape, np.nan)
poa_global_arr[valid_mask] = irradiance_tilted['poa_global']

In [16]:
meanL = []
for count, e5 in enumerate(era_time):
    if e5.day == doy:
        ssrd_hour = era_ds.GetRasterBand(count).ReadAsArray()
        if np.sum(ssrd_hour) == 0:
            continue
        else:
            if count > 606:
                break
            time_hour = np.full((2250000,), e5)
            # get solar viewing conditions for acquisition time
            
            solpos = solarposition.get_solarposition(
                time=time_hour,       # vector of timestamps
                latitude=lat_flat_masked, # vector of latitudes (same length as time or broadcastable)
                longitude=lon_flat_masked,
                altitude=dem_flat_masked
            )

            print(solpos['zenith'])
            # Compute clearness index
            ghi = ssrd_hour / 3600
            print(ghi)
            print(e5)
            # Decompose GHI to DNI and DHI using Erbs model
            dni_dhi = pvlib.irradiance.erbs(ghi.ravel(), solpos['zenith'], e5)
            dni, dhi = dni_dhi['dni'], dni_dhi['dhi']
            print(dni)

2019-07-26 05:00:00    77.952660
2019-07-26 05:00:00    77.952489
2019-07-26 05:00:00    77.952317
2019-07-26 05:00:00    77.952145
2019-07-26 05:00:00    77.951974
                         ...    
2019-07-26 05:00:00    77.776048
2019-07-26 05:00:00    77.775877
2019-07-26 05:00:00    77.775705
2019-07-26 05:00:00    77.775535
2019-07-26 05:00:00    77.775363
Name: zenith, Length: 2250000, dtype: float64
[[6.319133  6.3197    6.3202662 ... 7.16779   7.1683908 7.1689925]
 [6.318756  6.3193226 6.3198895 ... 7.167408  7.168009  7.1686106]
 [6.318379  6.3189454 6.3195124 ... 7.167026  7.167627  7.1682286]
 ...
 [5.770111  5.7706685 5.7712255 ... 6.6026177 6.6031933 6.603769 ]
 [5.769769  5.7703266 5.770883  ... 6.60225   6.602825  6.6034007]
 [5.769428  5.7699842 5.7705417 ... 6.6018815 6.602457  6.603032 ]]
2019-07-26 05:00:00
[0.06233883 0.06234828 0.0623577  ... 0.06611559 0.0661253  0.06613498]
2019-07-26 06:00:00    69.093295
2019-07-26 06:00:00    69.093117
2019-07-26 06:00:00    69

In [27]:
solpos['zenith'].shape

(2250000,)

In [3]:
irradiance_tilted = get_total_irradiance(
    surface_tilt=slope_flat_masked,
    surface_azimuth=aspect_flat_masked,
    dni=dni,
    ghi=ghi.ravel(),
    dhi=dhi,
    solar_zenith=solpos['zenith'],
    solar_azimuth=solpos['azimuth']
)

NameError: name 'slope_flat_masked' is not defined

In [ ]:

variable = 'geopotential'
    # for year in range(2017, 2025, 1):
year = 2019
files = [file for file in getFilelist(os.path.join(base_path, variable), '.grib') if str(year) in file]
        # for file in files:
file = files[5]
arr = stackReader(file)

In [ ]:
variables

In [ ]:
_, counts = np.unique(arr, axis=2, return_counts=True)
if len(counts) == 1:
    print(f'{variable} contains only a single value in year {year}')

In [ ]:
for variable in variables:
    for year in range(2017, 2025, 1):
        files = [file for file in getFilelist(os.path.join(base_path, variable), '.grib') if str(year) in file]
        for file in files:
            arr = stackReader(file)
            _, counts = np.unique(arr, axis=2, return_counts=True)
            if len(counts) == 1:
                print(f'{variable} contains only a single value in year {year} and month {INT_TO_MONTH[f'{int(file.split('_')[-1].split('.')[0]):02d}']}')
            else:
                print(f'{variable} contains mucho data in year {year} and month {INT_TO_MONTH[f'{int(file.split('_')[-1].split('.')[0]):02d}']}')